In [3]:

from typing import TypedDict,List

from langgraph.graph import StateGraph, END
from langchain_groq import ChatGroq
from langgraph.prebuilt import ToolNode

from langchain_core.messages import HumanMessage, AIMessage, BaseMessage,ToolMessage

import json
import ast
from typing import Any

In [4]:
from __future__ import annotations

from typing import TypedDict, List, Dict, Any, Optional
from dataclasses import dataclass
import math

from langgraph.graph import StateGraph, END

In [5]:
class ScrumState(TypedDict, total=False):
    cahier_de_charge: str
    team: Dict[str, Any]  # {members:[...], sprint_capacity_points:int, sprint_length_days:int}

    requirements: List[Dict[str, Any]]
    product_backlog: List[Dict[str, Any]]
    refined_backlog: List[Dict[str, Any]]
    estimated_backlog: List[Dict[str, Any]]
    dependencies: List[Dict[str, Any]]

    sprint_backlogs: List[Dict[str, Any]]
    assignments: List[Dict[str, Any]]

    validation: Dict[str, Any]


In [6]:
import json
from typing import Dict, Any
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

def llm_json(prompt: str) -> Dict[str, Any]:
    response = llm.invoke([
        HumanMessage(content=prompt + "\n\nReturn ONLY valid JSON. No markdown.")
    ])

    text = response.content.strip()


    print("text is:"+text)

    # Optional: handle ```json blocks
    if text.find("```json")!=-1:
        start_index = text.find("```json")
        end_index = text.find("```", start_index + 7)
        text = text[start_index + 7:end_index]
        print("new text is:"+text)

    return json.loads(text)


In [7]:
def extract_requirements_node(state: ScrumState) -> ScrumState:
    spec = state["cahier_de_charge"]

    prompt = f"""
Extract requirements from this cahier de charge.

Return JSON with:
requirements: [
  {{
    "id": "R1",
    "type": "functional|nfr",
    "text": "...",
    "priority": "must|should|could",
    "notes": "optional"
  }}
]

SPEC:
{spec}
"""
    out = llm_json(prompt)
    #print("requirement out is:"+str(out))
    state["requirements"] = out["requirements"]
    return state


def generate_product_backlog_node(state: ScrumState) -> ScrumState:
    reqs = state["requirements"]

    prompt = f"""
Convert requirements into a Scrum Product Backlog.
please for all the field use " " as separator and don't use ' '

Return JSON with:
product_backlog: [
  {{
    "epic": "Epic name",
    "stories": [
      {{
        "id": "US1",
        "title": "...",
        "as_a": "...",
        "i_want": "...",
        "so_that": "...",
        "acceptance_criteria": ["..."],
        "required_skills": ["backend","frontend","devops","qa"]
      }}
    ]
  }}
]

REQUIREMENTS:
{reqs}
"""
    out = llm_json(prompt)

    #print("productBacklog out is:"+str(out))
    state["product_backlog"] = out["product_backlog"]
    return state


def refine_backlog_node(state: ScrumState) -> ScrumState:
    pb = state["product_backlog"]

    prompt = f"""
Refine the backlog using INVEST:
- split stories that are too big
- remove duplicates
- add missing acceptance criteria
Return JSON with:
refined_backlog: [
  {{
    "id": "US1",
    "title": "...",
    "description": "...",
    "acceptance_criteria": ["..."],
    "required_skills": ["backend","frontend","devops","qa"]
  }}
]


IMPORTANT:
- Output ONLY JSON.
- No python code.
- No markdown.
- No explanations.
PRODUCT_BACKLOG:
{pb}
"""
    out = llm_json(prompt)
    print("refined_backlog out is:"+str(out))
    state["refined_backlog"] = out["refined_backlog"]
    return state


def estimate_backlog_node(state: ScrumState) -> ScrumState:
    refined = state["product_backlog"]
    team = state["team"]

    prompt = f"""
Estimate each story using Fibonacci story points: 1,2,3,5,8,13,21.

Return JSON with:
{{
  "estimated_backlog": [
    {{
      "id": "...",
      "title": "...",
      "points": 1|2|3|5|8|13|21,
      "risk": "low|medium|high",
      "complexity": "low|medium|high",
      "required_skills": [...]
    }}
  ]
}}

IMPORTANT:
- Output ONLY JSON.
- No python code.
- No markdown.
- No explanations.

TEAM:
{team}

STORIES:
{refined}
"""

    out = llm_json(prompt)
    print("estimated_backlog out is:"+str(out))
    state["estimated_backlog"] = out["estimated_backlog"]
    return state


def map_dependencies_node(state: ScrumState) -> ScrumState:
    stories = state["estimated_backlog"]

    prompt = f"""
Detect dependencies between stories.

Return JSON with:
dependencies: [
  {{
    "from": "US1",
    "to": "US5",
    "type": "blocks"
  }}
]
STORIES:
{stories}
"""
    out = llm_json(prompt)
    print("dependencies out is:"+str(out))
    state["dependencies"] = out["dependencies"]
    return state



In [8]:
def sprint_planner_node(state: ScrumState) -> ScrumState:
    stories = state["estimated_backlog"]
    deps = state.get("dependencies", [])
    capacity = int(state["team"].get("sprint_capacity_points", 20))

    # Simple heuristic: order by dependencies first (very simplified)
    # (In production: topological sort)
    ordered = stories[:]  # assume already prioritized by LLM

    sprints = []
    current = {"sprint": 1, "items": [], "total_points": 0}

    for st in ordered:
        pts = int(st["points"])
        if current["total_points"] + pts > capacity and current["items"]:
            sprints.append(current)
            current = {"sprint": current["sprint"] + 1, "items": [], "total_points": 0}

        current["items"].append(st["id"])
        current["total_points"] += pts

    if current["items"]:
        sprints.append(current)

    state["sprint_backlogs"] = sprints
    return state


In [9]:
def contributor_assigner_node(state: ScrumState) -> ScrumState:
    team_members = state["team"]["members"]
    stories = {s["id"]: s for s in state["estimated_backlog"]}

    # Very simple skill matching:
    # assign 1 main person who matches most skills
    assignments = []

    for sprint in state["sprint_backlogs"]:
        for story_id in sprint["items"]:
            story = stories[story_id]
            req_skills = set(story.get("required_skills", []))

            best = None
            best_score = -1

            for m in team_members:
                skills = set(m.get("skills", []))
                score = len(req_skills.intersection(skills))
                if score > best_score:
                    best_score = score
                    best = m

            assignments.append({
                "story_id": story_id,
                "title": story.get("title", ""),
                "assigned_to": best["name"] if best else None,
                "reason": f"matched_skills={best_score}"
            })

    state["assignments"] = assignments
    return state


def validation_node(state: ScrumState) -> ScrumState:
    capacity = int(state["team"].get("sprint_capacity_points", 20))
    sprints = state["sprint_backlogs"]
    stories = {s["id"]: s for s in state["estimated_backlog"]}

    issues = []

    for sp in sprints:
        total = sum(int(stories[sid]["points"]) for sid in sp["items"])
        if total > capacity:
            issues.append({
                "type": "over_capacity",
                "sprint": sp["sprint"],
                "total_points": total,
                "capacity": capacity
            })

    # Example: detect too big stories
    for s in state["estimated_backlog"]:
        if int(s["points"]) >= 13:
            issues.append({
                "type": "story_too_big",
                "story_id": s["id"],
                "points": s["points"]
            })

    state["validation"] = {
        "ok": len(issues) == 0,
        "issues": issues
    }
    return state


In [10]:
def route_after_validation(state: ScrumState) -> str:
    if state["validation"]["ok"]:
        return "done"
    # if stories too big -> refine again
    for issue in state["validation"]["issues"]:
        if issue["type"] == "story_too_big":
            return "refine"
    # if over capacity -> plan again
    return "replan"


In [11]:
def build_scrum_graph():
    g = StateGraph(ScrumState)

    g.add_node("extract_requirements", extract_requirements_node)
    g.add_node("generate_product_backlog", generate_product_backlog_node)
    g.add_node("refine_backlog", refine_backlog_node)
    g.add_node("estimate_backlog", estimate_backlog_node)
    g.add_node("map_dependencies", map_dependencies_node)
    g.add_node("sprint_planner", sprint_planner_node)
    g.add_node("contributor_assigner", contributor_assigner_node)
    g.add_node("validation", validation_node)

    # Edges
    g.set_entry_point("extract_requirements")
    g.add_edge("extract_requirements", "generate_product_backlog")
    g.add_edge("generate_product_backlog", "estimate_backlog")

    g.add_edge("refine_backlog", "estimate_backlog")
    g.add_edge("estimate_backlog", "map_dependencies")
    g.add_edge("map_dependencies", "sprint_planner")
    g.add_edge("sprint_planner", "contributor_assigner")
    g.add_edge("contributor_assigner", "validation")

    # Conditional routing after validation
    g.add_conditional_edges(
        "validation",
        route_after_validation,
        {
            "done": END,
            "refine": "estimate_backlog",
            "replan": "sprint_planner"
        }
    )

    return g.compile()


In [12]:
if __name__ == "__main__":
    graph = build_scrum_graph()

    cahier_de_charge = """
📄 Cahier de charge — Mini App “Event”
Objectif

Créer une application web pour gérer des événements et les inscriptions.

Acteurs

Membre

Admin

Fonctionnalités

L’admin peut créer un événement (titre, date, capacité).

L’admin peut publier / supprimer un événement.

Le membre peut voir les événements publiés.

Le membre peut s’inscrire à un événement.

Le membre peut annuler son inscription.

Un membre ne peut pas s’inscrire si l’événement est complet.

Contraintes

Authentification par email + mot de passe

Backend: Spring Boot

DB: PostgreSQL
    """

    team = {
        "sprint_length_days": 14,
        "sprint_capacity_points": 20,
        "members": [
            {"name": "Sami", "role": "Backend", "skills": ["backend", "sql", "spring"]},
            {"name": "Ali", "role": "Frontend", "skills": ["frontend", "angular", "ui"]},
            {"name": "Mouna", "role": "DevOps", "skills": ["devops", "docker", "ci_cd"]},
            {"name": "Hela", "role": "QA", "skills": ["qa", "testing"]}
        ]
    }

    result = graph.invoke({
        "cahier_de_charge": cahier_de_charge,
        "team": team
    })

    print("Sprints:", result["sprint_backlogs"])
    print("Assignments:", result["assignments"])
    print("Validation:", result["validation"])

text is:Here are the requirements extracted from the cahier de charge in valid JSON format:

```json
{
  "requirements": [
    {
      "id": "R1",
      "type": "functional",
      "text": "L'admin peut créer un événement (titre, date, capacité).",
      "priority": "must"
    },
    {
      "id": "R2",
      "type": "functional",
      "text": "L'admin peut publier / supprimer un événement.",
      "priority": "must"
    },
    {
      "id": "R3",
      "type": "functional",
      "text": "Le membre peut voir les événements publiés.",
      "priority": "must"
    },
    {
      "id": "R4",
      "type": "functional",
      "text": "Le membre peut s'inscrire à un événement.",
      "priority": "must"
    },
    {
      "id": "R5",
      "type": "functional",
      "text": "Le membre peut annuler son inscription.",
      "priority": "must"
    },
    {
      "id": "R6",
      "type": "nfr",
      "text": "Un membre ne peut pas s'inscrire si l'événement est complet.",
      "priority": "